In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

In [3]:
#model
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), # optimization
             loss='sparse_categorical_crossentropy', # loss function
             metrics=['accuracy']) # metrics / accuracy

In [5]:
# Data Preprocess

train_paths = sorted(glob('/Users/sengjeawang/Desktop/data/dataset/cifar/train/*.png'))[:1000]
test_paths = sorted(glob('/Users/sengjeawang/Desktop/data/dataset/cifar/test/*.png'))[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [7]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)

In [8]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.
    
    label = get_label(path)
    return image, label

In [9]:
def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

In [13]:
# Checkpoint

save_path = 'checkpoints'

In [14]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max') 

In [15]:
# Training

steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[checkpoint]
)

/Users/sengjeawang/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
31/31 [==============================] - 3s 85ms/step - loss: 2.3084 - accuracy: 0.0981 - val_loss: 2.2747 - val_accuracy: 0.1159

Epoch 00001: val_accuracy improved from -inf to 0.11593, saving model to checkpoints
INFO:tensorflow:Assets written to: checkpoints/assets
Epoch 2/20
31/31 [==============================] - 2s 75ms/step - loss: 2.2201 - accuracy: 0.1632 - val_loss: 2.2165 - val_accuracy: 0.1623

Epoch 00002: val_accuracy improved from 0.11593 to 0.16230, saving model to checkpoints
INFO:tensorflow:Assets written to: checkpoints/assets
Epoch 3/20
31/31 [==============================] - 2s 77ms/step - loss: 2.1613 - accuracy: 0.1601 - val_loss: 2.0535 - val_accuracy: 0.2228

Epoch 00003: val_accuracy improved from 0.16230 to 0.22278, saving model to checkpoints
INFO:tensorflow:Assets written to: checkpoints/assets
Epoch 4/20
31/31 [==============================] - 2s 75ms/step - loss: 2.0770 - accuracy: 0.2258 - val_loss: 1.9846 - val_accuracy: 0.2611

Epoch 000